<a href="https://colab.research.google.com/github/Avnish3909/Upwork-Job-scraper/blob/main/Upwork_Job_Scraper_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [31]:
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from csv import writer
import time

def scrape_upwork_page(url):
    chrome_options = Options()
    chrome_options.add_argument('--ignore-ssl-errors=yes')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)

    job_titles = driver.find_elements(By.CSS_SELECTOR, "h2[class='my-0 p-sm-right job-tile-title'] > a")
    job_pay = driver.find_elements(By.CSS_SELECTOR, "strong[data-test='job-type']")
    job_description_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-test='job-description-line-clamp'] span[data-test='job-description-text']")

    job_data = []

    for i in range(len(job_titles)):
        job_title = job_titles[i].text
        job_pay_text = job_pay[i].text
        job_description = job_description_elements[i].text
        job_data.append([job_title, job_pay_text, job_description, url])

    driver.quit()

    return job_data

if __name__ == '__main__':
    keyword = 'python'
    urls = [f'https://www.upwork.com/nx/jobs/search/?q={keyword}&sort=recency' for _ in range(4)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(scrape_upwork_page, urls))

    csv_filename = 'UpworkData.csv'
    with open(csv_filename, 'a', newline='') as fd:
        csv_file = writer(fd, delimiter=",")

        for job_data in results:
            for row in job_data:
                csv_file.writerow(row)


In [32]:
import pandas as pd

df= pd.read_csv("UpworkData.csv")
print(df.sample(5))

                             Provider Data Analyst II           Hourly  \
5   Build a Serverless Voice Chatbot/IVR Tree usin...  Hourly: $70-$85   
2       Seeking Software Developer for Customized App  Hourly: $45-$70   
28  Articulate entry-level DevOps Engineer to join...  Hourly: $70-$90   
33                        Work from home Data Analyst           Hourly   
9                            Provider Data Analyst II           Hourly   

   Provider Data Analyst II Analyze, research, and input relevant data for providers added to the organization's network through individual contracts, facility/ancillary contracts and/or newly acquired group contracts. Maintain changes to current provider files including provider...  \
5   I have already built a prototype on Twilio. It...                                                                                                                                                                                                                       